## Load packages

In [1]:
import pandas as pd
import numpy as np
import random
import re
import os

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"



## Load and inspect data 

In [2]:
sentencing_data_cleaned = pd.read_csv("../Data/sentencing_cleaned.csv", low_memory = False)

sentencing_data_cleaned.head()
sentencing_data_cleaned.shape
sentencing_data_cleaned.info

,CASE_ID,CASE_PARTICIPANT_ID,RECEIVED_DATE,OFFENSE_CATEGORY,PRIMARY_CHARGE_FLAG,CHARGE_ID,CHARGE_VERSION_ID,DISPOSITION_CHARGED_OFFENSE_TITLE,CHARGE_COUNT,DISPOSITION_DATE,...,simplified_offense_derived,is_black_derived,is_hisp_derived,is_white_derived,is_other_derived,is_male_derived,age_derived,sentenceymd_derived,sentenceym_derived,judgeid_derived
0,149765331439,175691153649,8/15/1984 12:00:00 AM,PROMIS Conversion,True,50510062193,112898098217,FIRST DEGREE MURDER,1,12/17/2014 12:00:00 AM,...,Homicide,True,False,False,False,True,27.0,2014-10-16,2014-10-01,judge_40
1,150065796098,162105612284,8/23/1984 12:00:00 AM,PROMIS Conversion,True,50792360681,113332130159,FIRST DEGREE MURDER,1,8/6/2014 12:00:00 AM,...,Homicide,True,False,False,False,True,30.0,2014-08-06,2014-08-01,judge_310
2,154954734978,225758446387,6/8/2001 12:00:00 AM,PROMIS Conversion,True,54885211141,174293345821,VIO BAIL BOND/CLASS 1,1,12/2/2013 12:00:00 AM,...,PROMIS Conversion,False,True,False,False,True,38.0,2013-12-02,2013-12-01,judge_162
3,155222744754,217349881776,1/31/2001 12:00:00 AM,PROMIS Conversion,True,53899906462,280120721775,POSS AMT CON SUB EXCEPT (A)/(D),1,9/10/2012 12:00:00 AM,...,Narcotics,True,False,False,False,False,33.0,2012-09-10,2012-09-01,judge_331
4,155327892699,217212381455,2/6/2001 12:00:00 AM,PROMIS Conversion,True,53938518259,164877860811,DUI LIC SUSPENDED OR REVOKED (EFFECTIVE 4-13-2...,1,9/19/2014 12:00:00 AM,...,PROMIS Conversion,False,False,True,False,True,49.0,2014-09-19,2014-09-01,judge_314


(135165, 52)

<bound method DataFrame.info of              CASE_ID  CASE_PARTICIPANT_ID          RECEIVED_DATE  \
0       149765331439         175691153649  8/15/1984 12:00:00 AM   
1       150065796098         162105612284  8/23/1984 12:00:00 AM   
2       154954734978         225758446387   6/8/2001 12:00:00 AM   
3       155222744754         217349881776  1/31/2001 12:00:00 AM   
4       155327892699         217212381455   2/6/2001 12:00:00 AM   
...              ...                  ...                    ...   
135160  445516248775         905289187184   2/3/2021 12:00:00 AM   
135161  445527409730         905322500577   2/4/2021 12:00:00 AM   
135162  445587767000         905518794790   2/9/2021 12:00:00 AM   
135163  445592613204         905533705601   2/9/2021 12:00:00 AM   
135164  445593200623         905535215556   2/9/2021 12:00:00 AM   

                                 OFFENSE_CATEGORY  PRIMARY_CHARGE_FLAG  \
0                               PROMIS Conversion                 True   
1  

In [3]:
# filter to incarcerated cases 
incarcerated_data = sentencing_data_cleaned.loc[(sentencing_data_cleaned["COMMITMENT_TYPE"] == "Illinois Department of Corrections")]

# Filter for only homicides (may switch this to another crime(s) with more disparity)
filtered_homicide = incarcerated_data[(incarcerated_data["simplified_offense_derived"] == "Homicide")].copy()

# group by race
judge_group = filtered_homicide.groupby("judgeid_derived").agg(is_black_derived=('is_black_derived', 'sum'), is_white_derived=('is_white_derived', 'sum')).reset_index()

# judges with greater than 25 cases for each race
judge_group_filtered = judge_group[(judge_group["is_black_derived"] >= 25) & (judge_group["is_white_derived"] >= 25)]



make_list = judge_group_filtered["judgeid_derived"].tolist()



judge_sentence_table = judge_group.pivot_table(values="COMMITMENT_TERM", index="judgeid_derived", columns=["is_black_derived", "is_white_derived"], aggfunc='mean', margins=True)

KeyError: 'COMMITMENT_TERM'

In [10]:
## Converting term length variables

## subsetting to just units of days, years, months, and life sentences


## Making variable for sentence length

sentencing_data_cleaned["term_length"] = (sentencing_data_cleaned.COMMITMENT_TERM + " " + sentencing_data_cleaned.COMMITMENT_UNIT)

sentencing_data_cleaned["term_length"].head

test = "3.0 Year(s)"

from dateutil.relativedelta import relativedelta

delta = relativedelta(years = int(test[0]))
delta

print(sentencing_data_cleaned[COMM])


sentencing_data_cleaned.COMMITMENT_UNIT.unique()

<bound method NDFrame.head of 0             62.0 Year(s)
1         1.0 Natural Life
2              3.0 Year(s)
3              2.0 Year(s)
4              2.0 Year(s)
                ...       
135160         2.0 Year(s)
135161         24.0 Months
135162           45.0 Days
135163           45.0 Days
135164           45.0 Days
Name: term_length, Length: 135165, dtype: object>

relativedelta(years=+3)

array(['Year(s)', 'Natural Life', 'Days', 'Months', 'Term', nan,
       'Dollars', 'Weeks', 'Pounds', 'Hours', 'Ounces'], dtype=object)